In [1]:
import weave
import litellm

In [2]:

# To drop unsupported openai params from the call, set `litellm.drop_params = True`
# Example: ChatGPT O-series models don't support temperature=0.

litellm.drop_params = True

In [3]:
completion = litellm.completion

In [4]:
weave.init("simple_prompting_guide")

Logged in as Weights & Biases user: bikashg.
View Weave data at https://wandb.ai/bikashg-spencer-group/simple_prompting_guide/weave


In [5]:
OPENAI_FAST_MODEL_NAME = "o4-mini"
OPENAI_SMART_MODEL_NAME = "gpt-4.1-mini"

In [6]:
@weave.op()
def get_completion(system_message: str, user_messages: list, model: str, **kwargs):
    # this fromat is specific to openai models only
    formatted_messages = [{"role": "system", "content": system_message}] + user_messages

    # Common arguments for the litellm completion function
    completion_args = {
        "model": model,
        "max_tokens": kwargs.pop('max_tokens', 4096),
        "temperature": kwargs.pop('temperature', 0),
        "messages": formatted_messages
    }
    completion_args.update(kwargs)  # Include any other additional arguments; as such

    # Generate and return the completion
    response = completion(**completion_args)
    return response.json()["choices"][0]["message"]["content"]

In [7]:
question = "Explain the latest prompting techniques and provide an example of each."

### Step 1: Raw Prompting

In [8]:
raw_prompt_response = get_completion(
    system_message= "",
    user_messages = [{"role": "user", "content": question}],
    model = OPENAI_FAST_MODEL_NAME
)

🍩 https://wandb.ai/bikashg-spencer-group/simple_prompting_guide/r/call/01968c58-838e-7183-bb9a-de055d68d95e
🍩 https://wandb.ai/bikashg-spencer-group/simple_prompting_guide/r/call/01968c58-d5fc-7231-819a-ded332d49179
🍩 https://wandb.ai/bikashg-spencer-group/simple_prompting_guide/r/call/01968c59-0f92-7612-b243-c57bcbcee6d0
🍩 https://wandb.ai/bikashg-spencer-group/simple_prompting_guide/r/call/01968c59-51a5-7980-9f5a-4839c4438bc1
🍩 https://wandb.ai/bikashg-spencer-group/simple_prompting_guide/r/call/01968c59-5195-7f01-a42d-88b23e3ae7eb
🍩 https://wandb.ai/bikashg-spencer-group/simple_prompting_guide/r/call/01968c59-51a1-70a3-a42f-4d701680d410
🍩 https://wandb.ai/bikashg-spencer-group/simple_prompting_guide/r/call/01968c59-8493-7460-8c81-4b756f0b9b43
🍩 https://wandb.ai/bikashg-spencer-group/simple_prompting_guide/r/call/01968c5f-545d-7aa3-838b-f26dd419b4e4
🍩 https://wandb.ai/bikashg-spencer-group/simple_prompting_guide/r/call/01968c76-40c8-78d1-a442-e11ebac5032e
🍩 https://wandb.ai/bikashg-s

In [9]:
print(raw_prompt_response[0:1000] + " ... ")

Here’s a survey of nine of the most-talked-about “next-gen” prompting patterns—what they are, why they help, and a minimal example for each.  

1. Zero-Shot Chain-of-Thought (CoT)  
  • Idea: Ask the model to “think step by step” even with no demonstrations.  
  • Why it helps: Elicits internal reasoning, reducing rash one-sentence answers.  
  • Prompt:  
    Q: “If there are 12 apples and I give half to a friend, how many are left?  
       Let’s think step by step.”  
  • Model output (abridg ... 


### Step 2. Prompting with Context

In [10]:
# The context we want to use here is the contents of the prompting guide file
PROMPT_GUIDE = "lilianweng_prompt_engineering.md"

def load_markdown_file(file_path: str) -> str:
    """
    Reads and returns the content of a markdown file specified by its path.

    Parameters:
        file_path (str): The path to the markdown file to be read.

    Returns:
        str: The content of the markdown file as a string.
    """
    with open(file_path, 'r', encoding='utf-8') as file:
        markdown_content = file.read()
    return markdown_content
context = load_markdown_file(PROMPT_GUIDE)

In [11]:
context_prompt_response = get_completion(
    system_message= "",
    user_messages = [{"role": "user", "content": context + "\n\nExplain the latest prompting techniques and provide an example of each"}],
    model = OPENAI_FAST_MODEL_NAME
)

In [12]:
print(context_prompt_response[0:1000] + " ... ")

Below is a concise tour of the most widely‐used modern prompting techniques, each with a minimal description and a toy prompt example. Feel free to adapt them to your own task.

1. Zero‐Shot Prompting  
   • Description: You describe the task and ask the model directly, with no demonstrations.  
   • Example:  
     ```
     Translate the following sentence into French.
     Input: “How are you today?”
     Output:
     ```

2. Few‐Shot Prompting  
   • Description: You prepend a handful of (inp ... 


### Step 3. Condition Responses with a System Prompt

In [13]:
system_message = """
Objective: Simplify prompt engineering concepts for easy understanding. Provide clear examples for each technique.
Tone: Friendly and educational, suitable for beginners.
Context: Assume basic AI knowledge; avoid deep technical jargon.
Guidance: Use metaphors and simple examples to explain concepts. Keep explanations concise and applicable.
Verification: Ensure clarity and relevance in responses, with practical examples.
Benefits: Help users grasp prompt engineering basics, enhancing their AI interaction experience.
"""

In [14]:
system_and_context_prompt_response = get_completion(
    system_message= system_message,
    user_messages = [{"role": "user", "content": context + "\n\nExplain the latest prompting techniques and provide an example of each"}],
    model = OPENAI_FAST_MODEL_NAME
)

In [15]:
print(system_and_context_prompt_response[0:1000] + " ... ")

Here’s a friendly, beginner-level tour of the main prompt-engineering tricks, each with a simple “real-world” metaphor and a tiny example you can try yourself.

1. Basic Prompting  
   • Zero-Shot (“Asking cold”)  
     Metaphor: You walk up to a friend and just ask, “Why is the sky blue?”  
     Example:  
       Prompt: “Explain why the sky is blue.”  
       Model: “Sunlight scatters in the atmosphere….”  
   • Few-Shot (“Showing examples”)  
     Metaphor: You show two photos of cats and dog ... 


### Step 4: System Prompts - Inputs. 

### Use f formatted strings to include placeholders in user message. 

In [16]:
user_message_prompt_template = "{input_context}\n\n{input_question}"

In [17]:
# To enable tracking of context and questions dynamically fed at runtime, we will write a function and decorate it with @weave.op
@weave.op()
def format_prompt(template_string: str, **kwargs):
    """
    Formats a prompt template with provided keyword arguments.

    This function takes a template string and a dictionary of keyword arguments,
    then formats the template string using these arguments.

    Parameters:
        prompt_template (str): The template string to be formatted.
        **kwargs (dict): Keyword arguments to format the template string with.

    Returns:
        str: The formatted prompt template.
    """
    return template_string.format(**kwargs)

In [18]:
# Just a simple check
format_prompt(template_string=user_message_prompt_template, input_context="test context", input_question="test question")

'test context\n\ntest question'

In [19]:
template_instantiated_user_message = format_prompt(template_string=user_message_prompt_template, input_context=context, input_question="""
Explain the differences between zero-shot, few-shot, and chain of thought 
prompting techniques? Please provide a clear explanation and a practical example 
for each technique within a structured format.
""")

In [20]:
system_and_template_instantiated_user_message_response = get_completion(
    system_message= system_message,
    user_messages = [{"role": "user", "content": template_instantiated_user_message}],
    model = OPENAI_FAST_MODEL_NAME
)

In [21]:
print(system_and_template_instantiated_user_message_response[0:1000] + " ... ")

Here’s a simple breakdown of three core prompting styles—zero-shot, few-shot, and chain-of-thought—each with a friendly metaphor, a plain-English explanation, and a hands-on example.

1. Zero-Shot Prompting  
   • Metaphor: Asking a friend a question “cold,” with no hints.  
   • What it is: You give the model just your request (the “task”) and let it answer directly. You don’t supply any examples of how to do it first.  
   • Why it helps: Super quick—uses few tokens—and you don’t need to prepa ... 


### Step 5: System Prompts - Outputs

In this step, we focus on improving the consistency and structure of our model's outputs by modifying the prompt template. By including specific tags and formatting instructions in the prompt, we can guide the model to respond in a way that is easier to parse and process. Get the ouput in XML, JSON etc. GPT prefers JSON.

In [22]:
system_msg_formatting_instruction = """
Format: Respond within a structured JSON object, using the keys provided in the prompt to organize your response.
Provide a condensed answer under the 'condensed_answer' key, detailed explanations under 'explanation' keys, 
and examples under 'example' keys within each explanation.
"""

user_msg_formatting_instruction = """
You must respond in JSON format.
Your response should follow this structure:
{{ 
  "answer": {{
    "condensed_answer": "CONDENSED_ANSWER",
    "explanation_1": {{
      "detail": "EXPLANATION_1",
      "example": "EXAMPLE_1"
    }},
    "explanation_2": {{
      "detail": "EXPLANATION_2",
      "example": "EXAMPLE_2"
    }},
    ...
  }}
}}
"""

In [23]:
def get_updated_prompt_with_output_formatting_instructions(in_system_message: str, in_user_message: str):
    return in_system_message + "\n" + system_msg_formatting_instruction, in_user_message + "\n" + user_msg_formatting_instruction

In [24]:
output_formatted_system_message, output_formatted_user_message = get_updated_prompt_with_output_formatting_instructions(system_message, template_instantiated_user_message)

In [25]:
output_formatted_system_and_user_message_response = get_completion(
    system_message= output_formatted_system_message,
    user_messages = [{"role": "user", "content": output_formatted_user_message}],
    model = OPENAI_FAST_MODEL_NAME,
    response_format={"type": "json_object"}  # An additional thing here is that we need to specify this.
)

In [26]:
print(output_formatted_system_and_user_message_response[0:1000] + " ... ")

{"answer":{"condensed_answer":"Zero-shot: ask the model directly without examples. Few-shot: show a few examples to guide the model. Chain-of-Thought: prompt the model to explain its reasoning step by step before answering.","explanation_1":{"detail":"Zero-shot prompting is like asking a friend a question without any context or samples. You state the task clearly and trust the model’s pre-trained knowledge to answer.","example":"Example: Translate to French:\n\"The weather is nice today.\""},"ex ... 


## Advanced Prompting Techniques

### Zero-shot Prompting

In [27]:
# All the above are examples of zero shot prompting. We didn't show any examples of any output (tone, style, etc.) we would like.

### Few-shot Prompting

In [28]:
few_shot_examples = """
Here are a few examples of prompting techniques in JSON format:
{{
    "answer": {{
        "condensed_answer": "Different prompting techniques are used to guide language models in generating desired outputs.",
        "explanation_1": {{
            "detail": "Translation prompts provide the model with a source language text and request the translation in a target language.",
            "example": "Translate the following English text to French: 'Hello, how are you?'"
        }},
        "explanation_2": {{
            "detail": "Sentiment classification prompts ask the model to determine the sentiment expressed in a given text.",
            "example": "Classify the sentiment of the following text: 'The movie was terrible.'"
        }},
        "explanation_3": {{
            "detail": "Factual question prompts require the model to provide an answer along with an explanation or reasoning.",
            "example": "What is the capital of Germany? Explain your reasoning."
        }}
    }}
}}"""

In [33]:
few_shot_user_message = few_shot_examples + "\n" + output_formatted_user_message

In [35]:
# print(few_shot_user_message)

In [36]:
few_shot_response = get_completion(
    system_message=output_formatted_system_message,
    user_messages=[{"role": "user", "content": few_shot_user_message}],
     model = OPENAI_FAST_MODEL_NAME,
    response_format={"type": "json_object"} # Comment this out for `Claude` models or `litellm.drop_params=True``
)

In [37]:
print(few_shot_response[0:1000] + " ... ")

{
  "answer": {
    "condensed_answer": "Zero-shot asks directly with no examples, few-shot shows a handful of examples to guide the model, and chain-of-thought asks the model to explain its reasoning step by step before answering.",
    "explanation_1": {
      "detail": "Zero-shot prompting gives the model a task instruction without any demonstrations. Imagine asking someone to solve a puzzle without showing any solved examples first.",
      "example": "Prompt: \"How many pencils do you have  ... 


### Chain of Thought

Note: We do not use the output formatting in this case as it will negate the chain of thought to instead enforce the formatting. It is important to explicitly incorporate the thought process desired in the prompt.

In [49]:
chain_of_thought_system_format = "Format: You must explicitly define the thought process and knowledge from the context to come to your conclusion for the question."
chain_of_thought_instruction = "Let's explicitly think step by step. My thought process is:\n"

In [50]:
chain_of_thought_system_message = system_message + "\n" + chain_of_thought_system_format
print(chain_of_thought_system_message)


Objective: Simplify prompt engineering concepts for easy understanding. Provide clear examples for each technique.
Tone: Friendly and educational, suitable for beginners.
Context: Assume basic AI knowledge; avoid deep technical jargon.
Guidance: Use metaphors and simple examples to explain concepts. Keep explanations concise and applicable.
Verification: Ensure clarity and relevance in responses, with practical examples.
Benefits: Help users grasp prompt engineering basics, enhancing their AI interaction experience.

Format: You must explicitly define the thought process and knowledge from the context to come to your conclusion for the question.


In [51]:
chain_of_thought_user_message = template_instantiated_user_message + "\n" + chain_of_thought_instruction
print(" ... "+ chain_of_thought_user_message[-500:])

 ... t support tool use in a chain (i.e. using the output of one tool as an input for another tool) or in an interactive way (i.e. adopt API response after human selection). Both are interesting future directions to expand the model for.


Explain the differences between zero-shot, few-shot, and chain of thought 
prompting techniques? Please provide a clear explanation and a practical example 
for each technique within a structured format.

Let's explicitly think step by step. My thought process is:



In [54]:
chain_of_thought_response = get_completion(
    system_message=chain_of_thought_system_message,
    user_messages=[{"role": "user", "content": chain_of_thought_user_message}],
     model = OPENAI_FAST_MODEL_NAME
)

In [55]:
print(chain_of_thought_response[0:1000] + " ... ")

Thought Process (concise summary):
- I know from the context that zero-shot means “no examples,” few-shot means “some examples,” and chain-of-thought (CoT) means “ask the model to show its reasoning steps.”
- I’ll pick a simple math problem (“2+2”) to illustrate all three, because arithmetic is familiar and keeps the focus on the prompting style rather than domain details.
- I’ll structure each technique into: Definition, Metaphor, and Practical Example.

Answer:

1. Zero-Shot Prompting  
   • D ... 
